# Economic activity data georeferenced
This notebook builds up on the previous one in which a non-spatial data set of countries admin 1 and 0 level is combined from DOSE and WDI. 

Here the geometries are added.

## Data

Reading in the data set that is generated from *missing_countries.ipynb*

In [3]:
import json
import io
import re
import itertools as iter
import numpy as np
import os

# data
import pandas as pd
import geopandas as gpd
import ibis as ib
from ibis import _
ib.options.interactive = True

from parameters import year

# plots
from datashader import transfer_functions as tf, reductions as rd
import pypalettes as pypal
import pydeck as pdk
from seaborn import color_palette

In [ ]:
# ddb.connect()
conn = ib.connect('duckdb://')
conn.list_tables() # empty

In [5]:
# boundaries = gpd.read_file("datasets/boundaries/GADM/gadm_410.gpkg")
# load boundaries data set directly into duckdb
# load extension first

res = conn.raw_sql("""INSTALL spatial;LOAD spatial;""")

In [6]:
# the merged file

#local path with folder where the downloaded shapefiles are stored 
#(both GADM and the custom one)

gadm_path = '../datasets/DOSE/DOSE_replication_files/DOSE_replication_files/Data/spatial data/' # ../../../../../

# Read shapefiles

file_name = "gadm36_1"


In [7]:

# gpd.list_layers(gadm_path+file_name+".gpkg")


In [8]:
gadm = gpd.read_file(gadm_path+"gadm36_levels_shp/" + file_name+".shp")
# has to be downloaded from https://gadm.org/download_world36.html; follow instructions in readme


In [ ]:
custom = gpd.read_file(gadm_path+'all_non_GADM_regions.shp')

#list of GADM countries whose data is not needed because we provide it with the custom file
unneeded_list = ["KAZ","MKD","NPL","PHL","LKA"]

#remove geometry for these countries from GADM
gadm_trim = gadm[~gadm.GID_0.isin(unneeded_list)]


# Merge/Combine multiple shapefiles into one
gadm_custom = gpd.pd.concat([gadm_trim, custom])
gadm_custom.drop(columns="fid",inplace=True)

gadm_custom.head()

In [10]:
#Export merged geodataframe into shapefile

out_path = "../datasets/DOSE/" # ../../../../../
gadm_custom.to_parquet(out_path+file_name+'_custom_merged.parquet')

# gpd.list_layers(gadm_path+"gadm_410-levels.gpkg")
# adm1 = gpd.read_file("../datasets/DOSE/DOSE_replication_files/DOSE_replication_files/Data/spatial data/gadm_410-levels.gpkg",layer="ADM_1")

In [ ]:
geoboundaries_file = '"../datasets/boundaries/GeoBoundaries/geoBoundariesCGAZ_ADM1.gpkg"'

gadm_file =  '"../datasets/boundaries/GADM/gadm_410.gpkg"'

dose_spatial_file = out_path+file_name+'_custom_merged.parquet'

conn.raw_sql(f"""CREATE OR REPLACE TABLE boundaries AS SELECT * FROM '{dose_spatial_file}';""")

# alternative method
# conn.read_geo("datasets/boundaries/GADM/gadm_410.gpkg", table_name="boundaries")


In [ ]:
# link the table from the duckdb, this is not performed by the previous operation
boundaries = conn.table("boundaries")
boundaries

In [13]:
# wb_countries = gpd.read_file("datasets/boundaries/WB_countries_Admin0_10m/WB_countries_Admin0_10m.shp")

### Reading the local dose-WDI data set


In [14]:
dose_wdi_path = "../datasets/local_data/dose-wdi/"
version = "0_3"

dose_light = conn.read_csv(source_list=f"{dose_wdi_path}{version}/dose_light_combined_{year}_{version}.csv",table_name="dose_light")


### Preparing the data

In [15]:
# nice function from ibis
boundaries = boundaries.rename("snake_case")
dose_light = dose_light.rename("snake_case")

In [ ]:
boundary_countries = conn.sql("Select distinct(gid_1) from boundaries;").to_pandas().iloc[:,0].to_list()
len(boundary_countries)

In [ ]:
boundaries.head()

In [18]:
def head_rand(conn: ib.backends.duckdb.Backend, table: str, limit:[int,str]=5):
    query = f"select * from {table} order by random() limit {limit};"
    # alternative using the duckdb sample command
    # query_ = f"select * from {table} using sample {limit};"
    return conn.sql(query=query)


In [ ]:
head_rand(conn=conn,table="dose_light",limit=10)


In [ ]:
head_rand(conn=conn,table="boundaries",limit=10)

In [21]:
# subsetting the boundaries data:
boundary_columns = ["GID_0","NAME_0","GID_1","NAME_1","geometry"]
boundary_columns = [str(x).lower() for x in boundary_columns]

boundaries = boundaries.select(boundary_columns)

In [ ]:
# head_rand(conn=conn,table="boundaries",limit=10)
boundaries.count()

In [ ]:
boundaries.gid_0.value_counts().execute().head(5)
# boundaries.filter(_.name_0=="Colombia"
#                   ).execute().tail(20)

In [ ]:
dose_light[dose_light.gid_0=="VEN"].execute()

In [ ]:
conn.list_tables()

In [26]:
# # grouping geometries query
# # first(country) as country
# #                 ,
# geom_query = """create view boundaries_1 as (select 
#                 first(gid_0) as gid_0
#                 ,first(name_0) as name_0
#                 ,first(gid_1) as gid_1
#                 ,first(name_1) as name_1 
#                 ,ST_Union_Agg(geom) as geom from boundaries group by gid_1);"""

# boundaries_1 = conn.raw_sql(geom_query)

In [27]:
# boundaries_1 = conn.table("boundaries_1")
# print(boundaries_1.nunique())
# boundaries_1

In [28]:
# boundaries_1_file = "boundaries_admin1.parquet"

# if not os.path.exists(boundaries_1_file):
#     boundaries_1.to_parquet("boundaries_admin1.parquet")
# else : 
#     conn.raw_sql(f"CREATE OR REPLACE TABLE boundaries_1 AS (SELECT * FROM '{boundaries_1_file}')")

In [29]:
# boundaries_0 = boundaries.select(_.gid_0) # .sql("select * EXCLUDE (geometry), ST_Centroid(ST_GeomFromWKB(geometry)) as geom from boundaries;")

In [30]:
boundaries_1 = conn.sql("""select * EXCLUDE (geometry)
                        , ST_Centroid(ST_GeomFromWKB(geometry)) as centr
                        , ST_GeomFromWKB(geometry) as geometry 
                        from boundaries;""")

In [ ]:
boundaries_1.head()

In [ ]:
boundaries_1.filter(_.GID_0=="LAO")

### Working only on centroids

In [33]:
# # grouping geometries query
# geom_centroid_query = """select country as country
#                 ,gid_0 as gid_0
#                 ,name_0 as name_0
#                 ,gid_1 as gid_1
#                 ,name_1 as name_1 
#                 ,ST_Centroid(geom) as geom from boundaries;"""
# boundaries_centr = conn.sql(geom_centroid_query)

In [34]:
boundaries_centr = boundaries_1.rename("snake_case").execute()

In [ ]:
boundaries_centr.loc[boundaries_centr.gid_0=="PER"]

In [ ]:
# regions to merge with DOSE

boundaries_1_centr = boundaries_centr.dissolve("gid_1")
boundaries_1_centr["centr"] = boundaries_1_centr.geometry.centroid
# 
# boundaries_1_centr.drop(columns="geom",inplace=True)
boundaries_1_centr.reset_index(inplace=True,drop=False)
# 
boundaries_1_centr = boundaries_1_centr.astype({"gid_1":str})
boundaries_1_centr.dtypes

In [37]:
gadm_gid_0_filename = f"{out_path}gadm_gid_0.parquet"


In [38]:

if not os.path.exists(gadm_gid_0_filename):
    boundaries_0_centr = conn.sql(
    """SELECT 
            gid_0, 
            ST_Union_Agg(geometry) as geometry 
            from 
                (select * EXCLUDE (geometry)
                            , ST_Centroid(ST_GeomFromWKB(geometry)) as centr
                            , ST_GeomFromWKB(geometry) as geometry 
                            from boundaries) 
            GROUP BY gid_0;""").execute() # boundaries_1
    
    boundaries_0_centr.set_crs(epsg=4326,inplace=True)
    boundaries_0_centr.to_parquet(gadm_gid_0_filename)




In [39]:
boundaries_0_centr = gpd.read_parquet(gadm_gid_0_filename)


In [40]:
boundaries_0_centr.columns = [x.lower() for x in boundaries_0_centr.columns]

In [ ]:

# boundaries_0_centr = boundaries_centr[["gid_0","geometry"]].set_geometry("geometry").set_crs(epsg=4326).dissolve(by="gid_0")
boundaries_0_centr["centr"] = boundaries_0_centr.geometry.centroid
# 
# boundaries_0_centr.drop(columns="geom",inplace=True)
boundaries_0_centr.reset_index(inplace=True,drop=False)
# 
boundaries_0_centr = boundaries_0_centr.astype({"gid_0":str})
boundaries_0_centr.dtypes

In [ ]:
boundaries_0_centr[boundaries_0_centr.gid_0=="SEN"].set_geometry("geometry").set_crs(epsg=4326).explore()

In [ ]:
dose_light.count()

In [44]:
dose_light_geo_ = dose_light.execute()

In [ ]:
dose_light_geo_.dtypes

In [46]:
dose_light_geo_ = dose_light_geo_.astype({"gid_1" : str})

In [ ]:
dose_light_geo_[dose_light_geo_.gid_1=="LAO"]

In [ ]:
boundaries_1_centr.dtypes

In [49]:
dose_light_geo = gpd.GeoDataFrame(dose_light_geo_.merge(boundaries_1_centr[["gid_1","centr","geometry"]],on="gid_1",how="left"))

In [ ]:
dose_light_geo[dose_light_geo.gid_1=="LAO"]

In [ ]:
missing_geoms = dose_light_geo.centr.isna()
missing_geoms

In [52]:
missing_countries = dose_light_geo_.loc[missing_geoms,"gid_0"].to_list()


In [53]:
# "LAO" in missing_countries

In [54]:
boundaries_0_centr_missing = boundaries_0_centr[boundaries_0_centr.gid_0.isin(missing_countries)].set_index("gid_0")
dose_light_geo.set_index("gid_0",inplace=True)


In [ ]:
boundaries_0_centr_missing.head()

In [56]:
dose_light_geo.loc[missing_countries,["centr","geometry"]] = boundaries_0_centr_missing[["centr","geometry"]]
dose_light_geo.reset_index(inplace=True,drop=False)

In [ ]:
# dose_light_geo.set_geometry("geometry",inplace=True).set_crs(epsg=4326,inplace=True)
# dose_light_geo.set_crs(epsg=4326,inplace=True)

dose_light_geo.set_geometry("centr",inplace=True)
dose_light_geo.set_crs(epsg=4326,inplace=True)

In [ ]:
dose_light_geo.loc[dose_light_geo.gid_0=="PER"].set_geometry("geometry").set_crs(epsg=4326).explore()

### Merging 

## Plotting

### Set up color palette for the map.

In [ ]:
viridis = color_palette("viridis", as_cmap=True) # the sns function
viridis

In [ ]:
col_pal = pypal.load_cmap("Apricot",reverse=True)
col_pal.N

In [61]:
def cmap(input, palette):
    input = np.nan_to_num(input).tolist()
    m = np.max(input)
    l = palette.N
    print("Max input : {}, palette colors : {}".format(m,l))
    return [[int(255*j) for j in palette(int(x/m*l))] for x in input] #

In [62]:
# unique_vals = pd.Series(dose_light_geo["grp_usd_2015_dose"].unique())
# unique_vals

In [ ]:
cols = cmap(np.log1p(dose_light_geo["grp_usd_2015"]),palette=col_pal)
dose_light_geo["color"] = cols
dose_light_geo["radius"] = np.log1p(dose_light_geo["grp_usd_2015"])
dose_light_geo.head()

In [64]:
# dose_light_geo.loc[dose_light_geo.country=="Colombia"]
dose_light_geo = dose_light_geo.set_geometry("geometry")

In [65]:
dose_light_geo["x"]=dose_light_geo.centr.x
dose_light_geo["y"]=dose_light_geo.centr.y

In [66]:
# dose_light_geo[dose_light_geo.gid_0=="LAO"]

In [67]:
# contains some missing bits. 
# dose_wdi_geo_filename = "dose_wdi_geo"
# dose_light_geo.to_parquet("../datasets/local_data/dose-wdi/"+dose_wdi_geo_filename+".parquet")

#### Deck map

In [68]:
# viewport = pdk.data_utils.compute_view(points=compact_geo_downscaled[['x', 'y']], view_proportion=0.9)

viewport = pdk.ViewState(longitude=0,latitude=0,zoom=2)

In [69]:
# Deck map showing the combined layers for each variable

gdp_layer = pdk.Layer(
    "ScatterplotLayer",
    dose_light_geo[["country","x","y","color","radius","gid_1","services_usd_2015","manufacturing_usd_2015","agriculture_usd_2015","grp_usd_2015"]],
    pickable=True,
    extruded=False,
    filled=True,
    stroked=True,
    opacity=.6,
    get_radius=["radius"],
    radius_scale=2000,
    radius_min_pixels=1,
    radius_max_pixels=15,
    get_position = ["x","y"],
    # get_polygons = "geom",
    get_fill_color = "color", #"[255*log_value/20,100,120]",
    get_line_color= [255, 255, 255, 0],
    line_width_min_pixels=0,
    line_width_max_pixels=1,
    )


# h3_layer = pdk.Layer(
#     "H3HexagonLayer",
#     dose_light_geo[["gdp","hex","color"]],
#     pickable=True,
#     stroked=True,
#     filled=True,
#     opacity=.5,
#     extruded=False,
#     get_hexagon="hex",
#     get_fill_color= "color", #[230, 200, 180, 255],
#     get_line_color=[0, 0, 0, 0],
#     line_width_min_pixels=1,
# )


In [70]:
r = pdk.Deck(layers=[gdp_layer]
            ,initial_view_state=viewport
            ,tooltip={"html": """<h3>{country} : {gid_1}</h3> 
                      <p> Services :  {services_usd_2015} </p> 
                      <p> Manufacturing: {manufacturing_usd_2015} </p>
                      <p> Agriculture: {agriculture_usd_2015} </p>
                      <p> Total: {grp_usd_2015} </p>
                      """}
            ,
            # ,mapbox_key="MAPBOX_API_KEY"
            )

In [ ]:
r.to_html("../deck_maps/dose-wdi_geometries_vis.html",iframe_height=800)

### Other maps:

In [72]:
! export DASK_DATAFRAME__QUERY_PLANNING=False

In [ ]:
import datashader as ds 

cvs = ds.Canvas(plot_width=650, plot_height=400)
agg = cvs.polygons(dose_light_geo, geometry='geometry',agg=ds.any())
tf.shade(agg)